In [1]:
import pandas as pd
import pickle

In [2]:
train_path = '../../data/raw/train.csv'
val_path = '../../data/raw/dev.csv'

train = pd.read_csv(train_path)
validation = pd.read_csv(val_path)

In [4]:
test_path = '../../data/raw/test_no_label.csv'
test = pd.read_csv(test_path)

In [20]:
train.columns

Index(['ex_id', 'user_id', 'prod_id', 'rating', 'label', 'date', 'review'], dtype='object')

In [37]:
pd.set_option('display.max_colwidth', -1)
train[train['review'].str.contains('34289034289042390234890234')]['review']

/Users/chuamelia/anaconda/envs/py35/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


130068    Holy mother of God! The Redhead just made me get all blasphemous and stuff.   Have you TRIED that pretzel or fried chicken or THE DELECTABLE COMPLIMENTARY SUGAR COOKIE THEY FEED YOU? Obviously not, or you would be there right now instead of reading this. So I am pretty jealous of YOU right now, because you're probably about to eat at The Redhead while I am most likely stuck in an office 1,547 miles away. Yeah, sucks to be me (not really). The seating is tight, but that didn't bother me. The servers were friendly and the chicken was juicy while remaining crunchy on the outside. This calorie countin' girl totally consumed like 34289034289042390234890234 calories at The Redhead, and ya know what? IT WAS WORTH IT.
Name: review, dtype: object

In [11]:
pd.set_option('display.max_colwidth', -1)
train[train['review'].str.contains('极其糟糕的中餐馆')]['review']

/Users/chuamelia/anaconda/envs/py35/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


192411    Terrible Restaurant. Im giving a 1-star only because there's no negative options. We ordered via delivery. 1. the price they post online is not the real price they will charge you. when we pay for  the delivery, the total cost is $8 more. 2. we paid extra for brown rice but got white rice. when we called them, they refused to take the responsibility and said it was the delivery guy's problem, with terrible attitude. 3. the serving is tiny for the amount they charge you. a sauteed vegetable dish cost $15. three dish cost us $74. I've never given one-star review on Yelp but this Han Dynasty made me have to tell all of you that----DO NOT DO BUSINESS WITH THESE PEOPLE. 极其糟糕的中餐馆。我真想骂你负分滚粗。非常贵，乱收费，分量少，态度差。能想到一切的差事儿都给他们家-­-汉朝--包揽了。3小盘菜74美金。
Name: review, dtype: object

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

cnt_vectorizer = CountVectorizer(stop_words='english', binary=True, min_df=5)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', binary=True, min_df=5)
cnt_vectorizer.fit(train['review'])
tfidf_vectorizer.fit(train['review'])

cnt_X_train = cnt_vectorizer.transform(train['review'])
tfidf_X_train = tfidf_vectorizer.transform(train['review'])

cnt_X_dev = cnt_vectorizer.transform(validation['review'])
tfidf_X_dev = tfidf_vectorizer.transform(validation['review'])

In [25]:
Y_train = train['label']

#### Creating Lookups

In [3]:
base = '/Users/chuamelia/Google Drive/Spring 2020/Machine Learning/fake-review-detection-project/data/processed/dev/'

num_reviews_by_user = train.groupby(train['user_id'])['ex_id'].count().to_frame().reset_index()
num_reviews_by_user.columns = ['user_id', 'train_num_reviews']
# num_reviews_by_user.to_csv(base + 'train_num_reviews_by_user_ac4119.csv', index=False, sep=',')

In [10]:
dev_num_reviews_by_user = validation.groupby(validation['user_id'])['ex_id'].count().to_frame().reset_index()
dev_num_reviews_by_user.columns = ['user_id', 'dev_num_reviews']


In [6]:
test_num_reviews_by_user = test.groupby(test['user_id'])['ex_id'].count().to_frame().reset_index()
test_num_reviews_by_user.columns = ['user_id', 'test_num_reviews']


In [12]:
stg_user_lkp_1  = pd.merge(num_reviews_by_user, dev_num_reviews_by_user, on='user_id', how='outer')
stg_user_lkp_2 =  pd.merge(stg_user_lkp_1, test_num_reviews_by_user, on='user_id', how='outer').fillna(0)

In [14]:
stg_user_lkp_2['cumulative_total_train_dev_reviews'] = stg_user_lkp_2['train_num_reviews'] +\
stg_user_lkp_2['dev_num_reviews'] 

In [16]:
stg_user_lkp_2['cumulative_total_train_dev_test_reviews'] = stg_user_lkp_2['train_num_reviews'] +\
stg_user_lkp_2['dev_num_reviews'] + stg_user_lkp_2['test_num_reviews']

In [20]:
stg_user_lkp_2.to_csv(base + 'num_reviews_by_user.csv', index=False, sep=',')

In [21]:
num_reviews_by_prod = train.groupby(train['prod_id'])['ex_id'].count().to_frame().reset_index()
num_reviews_by_prod.columns = ['prod_id', 'train_num_reviews']

dev_num_reviews_by_prod = validation.groupby(validation['prod_id'])['ex_id'].count().to_frame().reset_index()
dev_num_reviews_by_prod.columns = ['prod_id', 'dev_num_reviews']

test_num_reviews_by_prod = test.groupby(test['prod_id'])['ex_id'].count().to_frame().reset_index()
test_num_reviews_by_prod.columns = ['prod_id', 'test_num_reviews']

stg_prod_lkp_1  = pd.merge(num_reviews_by_prod, dev_num_reviews_by_prod, on='prod_id', how='outer')
stg_prod_lkp_2 =  pd.merge(stg_prod_lkp_1, test_num_reviews_by_prod, on='prod_id', how='outer').fillna(0)

stg_prod_lkp_2['cumulative_total_train_dev_reviews'] = stg_prod_lkp_2['train_num_reviews'] +\
stg_prod_lkp_2['dev_num_reviews'] 

stg_prod_lkp_2['cumulative_total_train_dev_test_reviews'] = stg_prod_lkp_2['train_num_reviews'] +\
stg_prod_lkp_2['dev_num_reviews'] + stg_prod_lkp_2['test_num_reviews']

stg_prod_lkp_2.to_csv(base + 'num_reviews_by_prod.csv', index=False, sep=',')

In [22]:
stg_prod_lkp_2

,prod_id,train_num_reviews,dev_num_reviews,test_num_reviews,cumulative_total_train_dev_reviews,cumulative_total_train_dev_test_reviews
0,0,155,18,37.0,173,210.0
1,1,191,12,50.0,203,253.0
2,2,118,21,27.0,139,166.0
3,3,39,6,14.0,45,59.0
4,4,374,66,107.0,440,547.0
...,...,...,...,...,...,...
918,918,278,37,74.0,315,389.0
919,919,322,56,108.0,378,486.0
920,920,218,28,60.0,246,306.0
921,921,138,20,34.0,158,192.0


In [ ]:
len(train['review'])

In [ ]:
len(tfidf_vectorizer.idf_)

In [ ]:
term_freq = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer.idf_))

In [31]:
pd.DataFrame(tfidf_vectorizer.get_feature_names()).to_csv(base + 'ac4119_all_terms.csv', index=False, sep=',')

In [40]:
for x in tfidf_vectorizer.get_feature_names()[-175:]:
    print(x, term_freq[x])

zweigelts 12.739562905538694
zweite 12.739562905538694
zweiten 12.739562905538694
zwickl 12.739562905538694
zwiec 12.739562905538694
zwirner 12.739562905538694
zwoa 12.739562905538694
zx44 12.33409779743053
zydeco 11.823272173664538
zygote 12.739562905538694
zyoga 12.739562905538694
zyrtec 12.739562905538694
zywec 12.739562905538694
zyweic 12.33409779743053
zywick 12.739562905538694
zywiec 12.046415724978749
zz 12.33409779743053
zza 12.739562905538694
zzzing 12.739562905538694
zzzz 12.046415724978749
zzzzz 12.739562905538694
zzzzzz 11.823272173664538
zzzzzzzz 12.739562905538694
zzzzzzzzz 12.739562905538694
zzzzzzzzzz 12.33409779743053
ºf 11.823272173664538
ànd 12.739562905538694
álvaro 12.739562905538694
är 12.739562905538694
ça 11.823272173664538
çatlayana 12.739562905538694
çünkü 12.739562905538694
éclair 12.739562905538694
éclairs 12.739562905538694
élan 12.739562905538694
éloigné 12.739562905538694
élysées 11.640950616870585
élégante 12.739562905538694
émilion 12.739562905538694
én

In [20]:
term_freq['good']

1.8479105708926928